# Fits 文件的创建与写入

## 创建一个新的 Fits 文件

In [2]:
import numpy as np
from astropy.io import fits

# prepara data
data1 = np.arange(100.0)
data2 = np.array([x for x in range(30)])

# append the headers besides the basic ones.
header2 = fits.Header()
header2['1'] = 'Good'
header2.comments['1'] = 'well wavelength calibrated'
header2['2'] = 'Bad'
header2['3'] = 'Flux'
header2.add_blank(before='1')
header2.add_comment('-- The explanation of different flag values', before='1')

# create 2 HDU Objects
hdu1 = fits.PrimaryHDU(data1)  # Primary here means first and unique.
hdu2 = fits.ImageHDU(data2, name='flags', header=header2)

# write to the fits file
hdu = fits.HDUList([hdu1, hdu2])
hdu.writeto('new2.fits', overwrite=True)

 - `PrimaryHDU` 在 fits 标准里**有且只能有一个，且必须在第 0 个**，详情参考[这个回答](https://stackoverflow.com/questions/49999408/astropy-fits-with-multiple-headers)

In [3]:
with fits.open('new2.fits') as f:
    f.info()
    data = f[1].data
    header = f[1].header

Filename: new2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (100,)   float64   
  1  FLAGS         1 ImageHDU        12   (30,)   int64   


In [3]:
header

XTENSION= 'IMAGE   '           / Image extension                                
BITPIX  =                   64 / array data type                                
NAXIS   =                    1 / number of array dimensions                     
NAXIS1  =                   30                                                  
PCOUNT  =                    0 / number of parameters                           
GCOUNT  =                    1 / number of groups                               
                                                                                
COMMENT -- The explanation of different flag values                             
1       = 'Good    '           / well wavelength calibrated                     
2       = 'Bad     '                                                            
3       = 'Flux    '                                                            
EXTNAME = 'FLAGS   '           / extension name                                 

## 在 fits 文件里追加 HDU

`astropy` [文档](https://docs.astropy.org/en/stable/io/fits/index.html#save-file-changes)里写道

> Because HDUList acts like a list we can also append, for example, an ImageHDU to an already existing HDUList:

In [4]:
data3 = np.ones(28)
hdu3 = fits.ImageHDU(data3, name='append')
with fits.open('new2.fits') as hdu:
    hdu.append(hdu3)
    hdu.writeto('new2.fits', overwrite=True)

In [5]:
with fits.open('new2.fits') as f:
    f.info()
    data = f[1].data
    header = f[1].header

Filename: new2.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  PRIMARY       1 PrimaryHDU       5   (100,)   float64   
  1  FLAGS         1 ImageHDU        12   (30,)   int64   
  2  APPEND        1 ImageHDU         7   (28,)   float64   


In [28]:
all_hdu_names = [f[i].name for i in range(len(f))]
all_hdu_names

['PRIMARY', 'FLAGS']

In [29]:
'FLAGS' in all_hdu_names

True

## 修改 PrimaryHDU 的 name

In [5]:
from astropy.io import fits

with fits.open('new2.fits') as f:
     f[0].name = 'test'
     f.writeto('new3.fits')

In [7]:
with fits.open('new3.fits') as f:
     f.info()

Filename: new3.fits
No.    Name      Ver    Type      Cards   Dimensions   Format
  0  TEST          1 PrimaryHDU       6   (100,)   float64   
  1  FLAGS         1 ImageHDU        12   (30,)   int64   
  2  APPEND        1 ImageHDU         7   (28,)   float64   
